In [1]:
from bs4 import BeautifulSoup
import re
from tqdm import tqdm

# Extract Book Info

In [616]:
path = '../raw_data_sample/book_info'
book_ids, book_names, languages, authors, b_urls = [], [], [], [], []
missed = []
decode_err = []

for book_id in tqdm(range(1, 105)):
    try:
        html = open('{}/{}.html'.format(path, book_id), 'r', encoding='ISO-8859-1')
        soup = BeautifulSoup(html, features='lxml')
    except FileNotFoundError:
        # print('{} missed!'.format(book_id))
        missed.append(book_id)
        continue
    except UnicodeDecodeError:
        # print('{} decoding error'.format(book_id))
        decode_err.append(book_id)
        continue

    # book_name
    try:
        book_name = soup.find(name='th', text='Title')
        book_name = book_name.parent.td.get_text().strip().replace('\n', ', ')
    except AttributeError:
        book_name = 'Unknown'
    
    # author
    try:
        language = soup.find(name='th', text='Language')
        language = language.parent.td.get_text().strip().replace('\n', ', ')
    except AttributeError:
        author = 'Unknown'
    
    # language
    try:
        author = soup.find(name='th', text='Author')
        author = author.parent.td.get_text().strip().replace('\n', ', ')
    except AttributeError:
        author = 'Unknown'
    
    # b_url
    try:
        b_url = 'https:' + soup.find(text=re.compile('Read this book online')).parent['href']
    except AttributeError:
        b_url = 'Unknown'

    book_ids.append(book_id)
    book_names.append(book_name)
    languages.append(language)
    authors.append(author)
    b_urls.append(b_url)

100%|██████████| 104/104 [00:02<00:00, 46.49it/s]


## write to txt file

In [617]:
book_num = len(book_ids)
print(book_num)
with open('Books.txt', 'w') as file:
    file.write('book_id \t book_name \t author \t b_url \t language \n')
    for i in range(book_num):
        file.write('{0} \t {1} \t {2} \t {3} \t {4} \n'.format(
                    book_ids[i], book_names[i], authors[i], b_urls[i], languages[i])
                  )

104


# Extract Figure Info

In [627]:
path = '../raw_data_sample/book_content'
book_ids, chap_nums, p_urls, dep_words = [], [], [], []
missed = []
decode_err = []
valid_books = []

for book_id in tqdm(range(1, 105)):
# for book_id in range(1, 20):
    try:
        html = open('{}/{}.html'.format(path, book_id), 'r', encoding='ISO-8859-1')
        soup = BeautifulSoup(html, features='lxml')
    except FileNotFoundError:
        # print('{} missed!'.format(book_id))
        missed.append(book_id)
        continue
    except UnicodeDecodeError:
        # print('{} decoding error'.format(book_id))
        decode_err.append(book_id)
        continue
    
    chap_num = 0
    next_ele = soup.title
    # print(next_ele)
    # print(book_id)

    while next_ele != None:
        if next_ele.name in ['h2', 'h3']:
            text = next_ele.get_text().strip().lower().split()
            if 'chapter' in text and 'chapter' == text[0]:
                chap_num += 1
                # print(text, chap_num)
        
        if (chap_num > 0) and (next_ele.name == 'img'):
            alt = next_ele['alt'].strip().replace('\n', ', ')
            src = next_ele['src']
            backslash = b_urls[book_id-1].rfind('/')
            p_url = b_urls[book_id-1][:backslash] + '/' + src
            
            book_ids.append(book_id)
            chap_nums.append(chap_num)
            p_urls.append(p_url)
            dep_words.append(alt)
        
        next_ele = next_ele.next_element
    
    if chap_num > 0:
        valid_books.append(book_id)

print(len(valid_books))

100%|██████████| 104/104 [00:10<00:00, 10.16it/s]

27


## write to txt file

In [630]:
pic_num = len(book_ids)
print(pic_num)
with open('Pictures.txt', 'w') as file:
    file.write('book_id \t chap_num \t p_url \t desp_words \n')
    for i in range(pic_num):
        file.write('{0} \t {1} \t {2} \t {3} \n'.format(
                    book_ids[i], chap_nums[i], p_urls[i], dep_words[i])
                  )

486
